In [17]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from scipy.optimize import curve_fit
import datetime
from matplotlib.dates import date2num
import numpy as np
import plotly
import chart_studio
import chart_studio.plotly as py
from plotly.offline import plot, iplot

pacienti = pd.read_csv('pacienti_agg.csv', parse_dates=[0], dayfirst=True)
predictii = pd.read_csv('predictii_cumulat.csv', parse_dates=[0], dayfirst=True)

In [30]:
#Total cazuri
total = pacienti.merge(predictii, left_on='data', right_on='data', how='outer')
x = total['data']
y1 = total['cumulat']
y2 = total['t1']

fig = go.Figure(data=[
    go.Bar(name='Predictie', x=x, y=y2),
    go.Bar(name='Real', x=x, y=y1)
])

fig.update_layout(title="Numar total de cazuri")

fig.show()
#py.iplot(fig, filename="total_cazuri")
#pio.write_html(fig, file='infectii_totale.html', auto_open=True)

In [31]:
#Cazuri externe pe zi
x = total['data']
y = total['pe_zi']

fig = go.Figure(data=[
    go.Bar(x=x, y=y)
])
fig.update_layout(title="Cazuri noi pe zi")
fig.show()

#py.iplot(fig, filename="cazuri_noi_pe_zi")
#pio.write_html(fig, file='infectii_noi_zi.html', auto_open=True)

In [33]:
def sigmoid(x, a, b):
    return 1 / (1 + np.exp(a*x + b))

def logistic(x, x0,y0,c,k):
    return c / (1 + np.exp(-k*(x-x0))) + y0

def exponential(x, a, b, c):
    return a * np.exp(b * x) + c

def parabolic(x, a, b, c):
    return a*x**2 + b*x + c

def plot_prediction(func, pacienti, title, filename=None, divider=1000):
    forward = 14
    l = len(pacienti)
    x = np.linspace(1, l, l)
    y = list(i/divider for i in pacienti["cumulat"])

    popt, pcov = curve_fit(func, x, y, maxfev=10000)

    yp = pacienti['cumulat']

    x2 = np.linspace(1, l+forward, l+forward)
    date = pd.date_range(start='26-02-2020', periods=l+forward)

    fig = go.Figure(data=[
        go.Scatter(name='Predictie', x=date, y=divider*func(x2, *popt), mode='lines+markers'),
        go.Bar(name='Real', x=date, y=yp)
    ])
    fig.update_layout(title=title)
    
    if filename:
        #py.iplot(fig, filename=filename)
        pio.write_html(fig, file=filename+'.html', auto_open=True)
    else:
        fig.show()
    
def plot_predictions_history(func, pacienti, title, filename=None, divider=1000):
    forward = 14
    l = len(pacienti)
    date = pd.date_range(start='26-02-2020', periods=l+forward)
    x2 = np.linspace(1, l+forward, l+forward)

    figure_data = []
    for i in range(22, l+1):
        if divider == 10000 and i < 26:
            divider2 = 1000
        else:
            divider2 = divider
            
        workset = pacienti[:i]
        l_2 = len(workset)
        x_2 = np.linspace(1, l_2, l_2)
        y_2 = list(i/divider2 for i in workset["cumulat"])
        popt_2, pcov_2 = curve_fit(func, x_2, y_2, maxfev=10000)
        figure_data.append(go.Scatter(name=f'Predictie {date[i].day} Martie', x=date, y=divider2*func(x2, *popt_2)),)

    fig = go.Figure(data=figure_data)
    fig.update_layout(title=title)
    
    if filename:
        #py.iplot(fig, filename=filename)
        pio.write_html(fig, file=filename+'.html', auto_open=True)
    else:
        fig.show()

In [34]:
plot_prediction(logistic, pacienti, "Cazuri totale + predictie curenta", divider=10000)
plot_predictions_history(logistic, pacienti, "Evolutia predictiilor", divider=10000)

plot_prediction(exponential, pacienti, "Cazuri totale + predictie pesimista", divider=1)
plot_predictions_history(exponential, pacienti, "Evolutia predictiilor pesimiste", divider=1)

plot_prediction(parabolic, pacienti, "Cazuri totale + predictie parabolica", divider=1)
plot_predictions_history(parabolic, pacienti, "Evolutia predictiilor parabolice", divider=1)

#plot_prediction(sigmoid, pacienti, "Cazuri totale + predictie sigmoid", divider=1000)
#plot_predictions_history(sigmoid, pacienti, "Evolutia predictiilor sigmoid", divider=1000)

In [35]:
plot_prediction(logistic, pacienti, "Cazuri totale + predictie curenta", "predictie_curenta", divider=10000)
plot_predictions_history(logistic, pacienti, "Evolutia predictiilor", "evolutie_predictii", divider=10000)

plot_prediction(exponential, pacienti, "Cazuri totale + predictie pesimista", "predictie_pesimista", divider=1)
plot_predictions_history(exponential, pacienti, "Evolutia predictiilor pesimiste", "evolutie_predictii_pesimiste", divider=1)

plot_prediction(parabolic, pacienti, "Cazuri totale + predictie parabolica", "predictie_parabolica", divider=1)
plot_predictions_history(parabolic, pacienti, "Evolutia predictiilor parabolice", "evolutie_predictii_parabolice", divider=1)

#plot_prediction(sigmoid, pacienti, "Cazuri totale + predictie sigmoid", "predictie_sigmoid", divider=1000)
#plot_predictions_history(sigmoid, pacienti, "Evolutia predictiilor sigmoid", "evolutie_predictii_sigmoid", divider=1000)

In [37]:
l = len(pacienti)
reference_count = 26

x_reference = np.linspace(1, reference_count, reference_count)
y = list(i/1000 for i in pacienti[:reference_count]["cumulat"])

popt_e, pcov = curve_fit(exponential, x_reference, y)
popt_s, pcov = curve_fit(sigmoid, x_reference, y)
popt_p, pcov = curve_fit(parabolic, x_reference, y)
popt_l, pcov = curve_fit(logistic, x_reference, y)

x = np.linspace(1, l, l)
y_e = [1000*exponential(i, *popt_e) for i in x]
y_s = [1000*sigmoid(i, *popt_s) for i in x]
y_p = [1000*parabolic(i, *popt_p) for i in x]
y_l = [1000*logistic(i, *popt_l) for i in x]

yp = pacienti['cumulat']
date = pd.date_range(start='26-02-2020', periods=len(pacienti))

fig = go.Figure(data=[
    go.Scatter(name='Predictie realista', x=date, y=y_l, mode='lines+markers'),
    go.Scatter(name='Predictie pesimista', x=date, y=y_e, mode='lines+markers'),
    go.Scatter(name='Predictie parabolica', x=date, y=y_p, mode='lines+markers'),
    #go.Scatter(name='Predictie sigmoid', x=date, y=y_s, mode='lines+markers'),
    go.Bar(name='Real', x=date, y=yp)
])
fig.update_layout(title="Predictii din 22 Martie vs real")
fig.show()
#py.iplot(fig, filename="compara_predictii")
#pio.write_html(fig, file='compara_predictii.html', auto_open=True)

In [38]:
error_e = abs(yp - y_e)
error_s = abs(yp - y_s)
error_p = abs(yp - y_p)
error_l = abs(yp - y_l)

fig = go.Figure(data=[
    go.Bar(name='Predictie realista', x=date, y=error_l),
    go.Bar(name='Predictie pesimista', x=date, y=error_e),
    go.Bar(name='Predictie parabolica', x=date, y=error_p),
    #go.Bar(name='Predictie sigmoid', x=date, y=error_s)
])
fig.update_layout(title="Erori de predictie")
#fig.show()
#py.iplot(fig, filename="eroare_predictii")
pio.write_html(fig, file='eroare_predictii.html', auto_open=True)

In [39]:
decese = pd.read_csv('decese_agg.csv', parse_dates=[0], dayfirst=True)
x_d = decese['data']
y1_d = decese['cumulat']
#y2 = total['t1']

fig = go.Figure(data=[
    go.Bar(name='Real', x=x_d, y=y1_d)
])
fig.update_layout(title="Numar total de decese")
#fig.show()

pio.write_html(fig, file='decese.html', auto_open=True)

In [40]:
x = decese['data']
y = decese['pe_zi']

fig = go.Figure(data=[
    go.Bar(x=x, y=y)
])
fig.update_layout(title="Decese noi pe zi")
#fig.show()

pio.write_html(fig, file='decese_noi_zi.html', auto_open=True)

In [29]:
#plot_prediction(logistic, decese, "Decese totale + predictie", divider=10)
#100 * decese["cumulat"]
#decese["cumulat"] = decese["cumulat"].apply(lambda x: x * 100)
#decese
plot_prediction(logistic, decese, "Decese totale + predictie", divider=10000)

In [61]:
x_d = decese['data']
y1_d = decese['cumulat']
#y2 = total['t1']

fig = go.Figure(data=[
    go.Bar(name='Real', x=x_d, y=y1_d)
])
fig.update_layout(title="Numar total de decese")
fig.show()